# K-means Clustering

## Giới thiệu:

Trước hết tạo tâm cụm và dữ liệu cho từng cụm bằng cách lấy mẫu theo phân phối chuẩn có kỳ vọng là tâm của cụm đó và ma trận hiệp phương sai là ma trận đơn vị. Hàm `cdist` trong `scipy.spatial.distance` được dùng để tính khoảng cách giữa 2 điểm.

Dữ liệu được tạo bằng cách lấy ngẫu nhiên 500 điểm cho mỗi cụm với kỳ vọng lần lượt là (2, 2), (8, 3), (3, 6); ma trận hiệp phương sai là ma trận đơn vị

In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance  import cdist
import random

In [13]:
np.random.seed(18)
means = [[2, 2], [8, 3], [3, 6]] # tâm
cov = [[1, 0], [0, 1]] # hiệp phương sai
N = 500
X0 = np.random.multivariate_normal(means[0], cov, N)
X1 = np.random.multivariate_normal(means[1], cov, N)
X2 = np.random.multivariate_normal(means[2], cov, N)

X = np.concatenate((X0, X1, X2), axis = 0) # Kết hợp các mẫu thành một ma trận
K = 3 # 3 clusters
original_label = np.asarray([0]*N + [1]*N + [2]*N).T # Tạo mãng chứa nhãn gốc của mỗi mẫu dữ liệu. Cụm X0 có mẫu 0, ...

## Các hàm cần thiết cho thuật toán phân cụm **K-means**

Trước khi viết thuật toán chính ta cần một số hàm phụ trợ:

a. `kmeans_init_centroids` khởi tạo các tâm cụm. <br>
b. `kmeans_asign_label` tìm nhãn mới có các điểm khi biết tâm cụm<br>
c. `kmeans_update_centroids` cập nhật tâm mới sau khi biết nhãn từng điểm<br>
d. `has_converged` kiểm tra điều kiện dừng của thuật toán

In [16]:
def kmeans_init_centroids(X, k):
    # tìm ngẫu nhiên k hàng để làm tâm ban đầu cho thuật toán
    return X[np.random.choice(X.shape[0], k, replace=False)]

# Hàm này chọn ngẫu nhiên k hàng từ dữ liệu X để làm tâm ban đầu cho thuật toán.

def kmeans_assign_labels(X, centroids):
    # tính toán cặp điểm X_i và tâm m_j
    D = cdist(X, centroids)
    # trả về chỉ số của tâm gần nhất
    return np.argmin(D, axis=1)

# Hàm này tính toán khoảng cách giữa mỗi điểm dữ liệu X_i và tất cả các tâm m_j, sau đó gán nhãn cho mỗi điểm dữ liệu bằng chỉ số của tâm gần nhất.

def kmeans_update_centroids(X, labels, K):
    centroids = np.zeros((K, X.shape[1]))
    for k in range(K):
        # collect all points that are assigned to the k-th cluster
        Xk = X[labels == k, :]
        centroids[k,:] = np.mean(Xk, axis = 0) # take average
    return centroids

# Hàm này cập nhật vị trí của các tâm dựa trên dữ liệu đã được gán nhãn.
# Đối với mỗi cụm, nó tính trung bình cộng của tất cả các điểm dữ liệu trong cụm đó và đặt tâm của cụm đó tại vị trí trung bình này.

def has_converged(centroids, new_centroids):
    # Trả về True nếu cả 2 tâm đều như nhau
    return (set([tuple(a) for a in centroids]) == set([tuple(a) for a in new_centroids]))

# Hàm này kiểm tra xem thuật toán có đã hội tụ hay chưa. Nếu tất cả các tâm không thay đổi sau một vòng lặp, thì thuật toán đã hội tụ và hàm trả về True.

Phần chính của K-means

In [17]:
def kmeans(X, K):
    centroids = [kmeans_init_centroids(X, K)]
    labels = []
    it = 0
    while True:
        labels.append(kmeans_assign_labels(X, centroids[-1]))
        new_centroids = kmeans_update_centroids(X, labels[-1], K)
        if has_converged(centroids[-1], new_centroids):
            break
        centroids.append(new_centroids)
        it += 1
    return (centroids, labels, it)


Áp dụng thuật toán vào dữ liệu ban đầu:

In [18]:
centroids, labels, it = kmeans(X, K)
print('Centers found by our algorithm:\n', centroids[-1])

Centers found by our algorithm:
 [[3.02702878 5.95686115]
 [8.07476866 3.01494931]
 [1.9834967  1.96588127]]


### Sử dụng thư viện của `sk-learn`

In [15]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=3, random_state=0, n_init = 10).fit(X)
print('Centers found by scikit-learn:')
print(model.cluster_centers_)

pred_labels = model.predict(X)

Centers found by scikit-learn:
[[3.02801296 5.96081708]
 [8.07476866 3.01494931]
 [1.98459808 1.96989964]]


c:\Users\Thang\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Ta thấy kết quả rất gần với kỳ vọng